In [16]:
pip install pandas

Looking in indexes: https://mirror.nju.edu.cn/pypi/web/simpleNote: you may need to restart the kernel to use updated packages.



In [2]:
pip install numpy

Looking in indexes: https://mirror.nju.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install matplotlib

Looking in indexes: https://mirror.nju.edu.cn/pypi/web/simpleNote: you may need to restart the kernel to use updated packages.



In [4]:
pip install seaborn

Looking in indexes: https://mirror.nju.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings

In [11]:
warnings.filterwarnings('ignore')

In [12]:
# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体显示中文
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

In [13]:
df_train = pd.read_csv(r'H:\期末考试\可视化\data (1)\data\train.csv')
df_test = pd.read_csv(r'H:\期末考试\可视化\data (1)\data\test.csv')
df_stores = pd.read_csv(r'H:\期末考试\可视化\data (1)\data\stores.csv')
df_holidays = pd.read_csv(r'H:\期末考试\可视化\data (1)\data\holidays_events.csv')
df_oil = pd.read_csv(r'H:\期末考试\可视化\data (1)\data\oil.csv')
df_submission = pd.read_csv(r'H:\期末考试\可视化\data (1)\data\sample_submission.csv')

In [15]:
print("\n数据基本信息：")
for name, df in zip(['训练集', '测试集', '门店', '节假日', '油价', '提交样本'],
                    [df_train, df_test, df_stores, df_holidays, df_oil, df_submission]):
    print(f"\n{name}数据基本信息：")
    df.info()


数据基本信息：

训练集数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB

测试集数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB

门店数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null 

In [20]:
def preprocess_data():
    # 复制数据避免修改原始数据
    train = df_train.copy()
    test = df_test.copy()
    stores = df_stores.copy()
    holidays = df_holidays.copy()
    oil = df_oil.copy()

    # 转换日期格式
    for df in [train, test, holidays, oil]:
        df['date'] = pd.to_datetime(df['date'])

    # 处理油价缺失值（前向填充）
    oil['dcoilwtico'] = oil['dcoilwtico'].ffill()

    # 合并数据集
    merged = train.merge(stores, on='store_nbr', how='left')
    merged = merged.merge(holidays, on='date', how='left')
    merged = merged.merge(oil, on='date', how='left')

    # 处理合并后的缺失值
    merged['type_y'] = merged['type_y'].fillna('Normal Day')  # 非节假日标记
    merged['locale'] = merged['locale'].fillna('None')
    merged['locale_name'] = merged['locale_name'].fillna('None')
    merged['description'] = merged['description'].fillna('None')
    merged['transferred'] = merged['transferred'].fillna(False)

    # 提取日期特征
    merged['year'] = merged['date'].dt.year
    merged['month'] = merged['date'].dt.month
    merged['day'] = merged['date'].dt.day
    merged['dayofweek'] = merged['date'].dt.dayofweek
    merged['is_weekend'] = merged['dayofweek'].isin([5, 6])
    merged['quarter'] = merged['date'].dt.quarter

    # 标记是否为节假日
    merged['is_holiday'] = merged['type_y'] != 'Normal Day'

    # 创建季节特征
    merged['season'] = ((merged['month'] % 12 + 3) // 3).map({1: '春', 2: '夏', 3: '秋', 4: '冬'})

    # 处理油价特征
    merged['oil_price'] = merged['dcoilwtico']

    return merged


In [21]:
df = preprocess_data()

In [22]:
df.to_csv('preprocessed_data.csv', index=False)
print("\n数据预处理完成，已保存为 preprocessed_data.csv")


数据预处理完成，已保存为 preprocessed_data.csv


In [23]:
def statistical_analysis(df):
    # 1. 销售趋势分析
    daily_sales = df.groupby('date')['sales'].sum().reset_index()

    # 时间序列分解
    daily_sales.set_index('date', inplace=True)
    decomposition = seasonal_decompose(daily_sales['sales'], model='additive', period=30)
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid

    # 2. 节假日影响分析
    holiday_effect = df.groupby('type_y')['sales'].mean().sort_values(ascending=False).reset_index()

    # 3. 商品类别分析
    category_sales = df.groupby('family')['sales'].sum().sort_values(ascending=False).reset_index()

    # 4. 门店表现分析
    store_performance = df.groupby('store_nbr')['sales'].sum().sort_values(ascending=False).reset_index()
    store_types = df.groupby('type_x')['sales'].sum().sort_values(ascending=False).reset_index()

    # 5. 城市销售分布
    city_sales = df.groupby('city')['sales'].sum().sort_values(ascending=False).reset_index()

    # 6. 油价与销售关系
    oil_sales_corr = df[['sales', 'oil_price']].corr().iloc[0, 1]

    # 7. 时间序列预测 (ARIMA模型)
    # 准备数据
    train_size = int(len(daily_sales) * 0.8)
    train, test = daily_sales.iloc[:train_size], daily_sales.iloc[train_size:]

    # 训练ARIMA模型
    model = ARIMA(train['sales'], order=(5, 1, 0))  # 简单ARIMA模型，实际应通过AIC/BIC选择最优参数
    model_fit = model.fit()

    # 预测
    predictions = model_fit.forecast(steps=len(test))
    mse = mean_squared_error(test['sales'], predictions)
    rmse = np.sqrt(mse)

    # 未来30天销售预测
    future_predictions = model_fit.forecast(steps=30)

    return {
        'daily_sales': daily_sales,
        'holiday_effect': holiday_effect,
        'category_sales': category_sales,
        'store_performance': store_performance,
        'store_types': store_types,
        'city_sales': city_sales,
        'oil_sales_corr': oil_sales_corr,
        'time_series': {
            'trend': trend,
            'seasonal': seasonal,
            'residual': residual,
            'train': train,
            'test': test,
            'predictions': predictions,
            'future_predictions': future_predictions,
            'rmse': rmse
        }
    }

In [27]:
analysis_results = statistical_analysis(df)
print("\n统计分析完成")
print(f"油价与销售的相关系数: {analysis_results['oil_sales_corr']:.4f}")
print(f"时间序列预测的RMSE: {analysis_results['time_series']['rmse']:.4f}")


统计分析完成
油价与销售的相关系数: -0.0790
时间序列预测的RMSE: 256286.3974


### 销售趋势图

In [31]:
def create_visualizations(analysis_results, df):
    # 1. 销售趋势图
    plt.figure(figsize=(14, 7))
    plt.plot(analysis_results['daily_sales'], label='每日销售额')
    plt.title('销售趋势分析')
    plt.xlabel('日期')
    plt.ylabel('销售额')
    plt.legend()
    plt.tight_layout()
    plt.savefig("H:\期末考试\可视化\销售趋势图.png")
    plt.close()
create_visualizations(analysis_results, df)  # 确保 analysis_results 和 df 已定义

### 时间序列分解图

In [33]:
def create_visualizations(analysis_results, df):
    # 2. 时间序列分解图
    decomposition = analysis_results['time_series']
    plt.figure(figsize=(14, 10))

    plt.subplot(411)
    plt.plot(analysis_results['daily_sales'], label='原始数据')
    plt.legend(loc='best')
    plt.title('时间序列分解')

    plt.subplot(412)
    plt.plot(decomposition['trend'], label='趋势')
    plt.legend(loc='best')

    plt.subplot(413)
    plt.plot(decomposition['seasonal'], label='季节性')
    plt.legend(loc='best')

    plt.subplot(414)
    plt.plot(decomposition['residual'], label='残差')
    plt.legend(loc='best')

    plt.tight_layout()
    plt.savefig("H:\期末考试\可视化\时间序列分解图.png")
    plt.close()
create_visualizations(analysis_results, df)  # 确保 analysis_results 和 df 已定义

### 节假日影响条形图

In [35]:
def create_visualizations(analysis_results, df):
  
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False

    
    plt.figure(figsize=(12, 6))
    

    type_mapping = {
        'Additional': '附加',
        'Transfer': '调休',
        'Bridge': '桥梁日',
        'Event': '事件日',
        'Work_Day': '工作日',
        'Holiday': '节假日',
        'Normal_Day': '普通日'
    }
    

    holiday_data = analysis_results['holiday_effect'].copy()
    holiday_data['type_cn'] = holiday_data['type_y'].map(type_mapping)
    
    sns.barplot(x='type_cn', y='sales', data=holiday_data)
    plt.title('节假日类型对销售的影响')
    plt.xlabel('节假日类型')
    plt.ylabel('平均销售额')
    plt.xticks(rotation=35)  # 微调旋转角度
    plt.tight_layout()
    

    plt.savefig(r"H:\期末考试\可视化\节假日影响条形图.png")
    plt.close()


create_visualizations(analysis_results, df)

### 商品类别销售分布

In [38]:
def create_visualizations(analysis_results, df):
    # 设置中文字体
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False

    # 4. 商品类别销售分布（前10）
    plt.figure(figsize=(12, 8))
    
    # 创建中英文映射字典
    category_mapping = {
        'GROCERY': '杂货',
        'BEVERAGES': '饮料',
        'PRODUCE': '农产品',
        'CLEANING': '清洁用品',
        'DAIRY': '乳制品',
        'BREAD/BAKERY': '面包/烘焙',
        'POULTRY': '禽类',
        'MEATS': '肉类',
        'PERSONAL CARE': '个人护理',
        'DELI': '熟食'
    }
    
    # 处理数据副本
    category_data = analysis_results['category_sales'].head(10).copy()
    category_data['family_cn'] = category_data['family'].map(category_mapping)
    
    # 绘制水平条形图
    sns.barplot(x='sales', 
                y='family_cn', 
                data=category_data,
                color='darkblue')  # 统一深蓝色
    
    # 设置科学计数法格式
    plt.ticklabel_format(axis='x', style='sci', scilimits=(6,6))
    
    plt.title('商品类别销售分布（前10）')
    plt.xlabel('总销售额')
    plt.ylabel('商品类别')
    plt.tight_layout()
    
    # 修正路径格式
    plt.savefig(r"H:\期末考试\可视化\商品类别销售分布.png")
    plt.close()

# 调用函数
create_visualizations(analysis_results, df)

### 门店销售排行

In [41]:
def create_visualizations(analysis_results, df):
    plt.figure(figsize=(12, 8))
    sns.barplot(x='sales', y='store_nbr', data=analysis_results['store_performance'].head(10))
    plt.title('门店销售排行（前10）')
    plt.xlabel('总销售额')
    plt.ylabel('门店编号')
    plt.tight_layout()
    plt.savefig("H:\期末考试\可视化\门店销售排行.png")
    plt.close()
create_visualizations(analysis_results, df)

### 城市销售分布

In [43]:
def create_visualizations(analysis_results, df):
    # 设置中文字体（若之前未设置需保留此配置）
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False

    # 城市销售分布（前10）
    plt.figure(figsize=(12, 8))
    
    # 创建厄瓜多尔城市中英文映射字典
    city_mapping = {
        'Quito': '基多',
        'Guayaquil': '瓜亚基尔',
        'Cuenca': '昆卡',
        'Ambato': '安巴托',
        'Santo Domingo': '圣多明各',
        'Machala': '马查拉',
        'Cayambe': '卡扬贝',
        'Latacunga': '拉塔昆加',
        'Daule': '道勒',
        'Loja': '洛哈'
    }
    
    # 处理数据副本
    city_data = analysis_results['city_sales'].head(10).copy()
    city_data['city_cn'] = city_data['city'].map(city_mapping)
    
    # 绘制水平条形图
    sns.barplot(x='sales', 
                y='city_cn',  # 使用映射后的中文城市名
                data=city_data.sort_values('sales', ascending=False),  # 确保排序正确
                color='steelblue')  # 保持原图蓝色风格
    
    # 设置科学计数法（1e8显示）
    plt.ticklabel_format(axis='x', style='sci', scilimits=(8,8))
    
    plt.title('城市销售分布（前10）')
    plt.xlabel('总销售额')
    plt.ylabel('城市')
    plt.tight_layout()
    
    # 修正路径格式
    plt.savefig(r"H:\期末考试\可视化\城市销售分布.png")
    plt.close()

# 调用函数
create_visualizations(analysis_results, df)

### 销售预测图

In [55]:
def create_visualizations(analysis_results, df):
    train = analysis_results['time_series']['train']
    test = analysis_results['time_series']['test']
    predictions = analysis_results['time_series']['predictions']
    future_predictions = analysis_results['time_series']['future_predictions']

    plt.figure(figsize=(14, 7))
    plt.plot(train.index, train['sales'], label='训练数据')
    plt.plot(test.index, test['sales'], label='实际销售')
    plt.plot(test.index, predictions, label='预测销售', linestyle='--')
    # 修正路径格式
    plt.savefig(r"H:\期末考试\可视化\销售预测图.png")
    plt.close()

# 调用函数
create_visualizations(analysis_results, df)

### 未来预测图

In [63]:
def create_visualizations(analysis_results, df):
    # 从 analysis_results 中提取预测结果
    future_predictions = analysis_results["future_predictions"]
    
    future_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=30)
    plt.plot(future_dates, future_predictions, label='未来30天预测', linestyle=':', color='green')
    
    plt.title('销售预测')
    plt.xlabel('日期')
    plt.ylabel('销售额')
    plt.legend()
    plt.tight_layout()
    plt.savefig(r"H:\期末考试\可视化\未来预测图.png")  # 使用原始字符串避免转义错误
    plt.close()

# 调用函数（无需传递 future_predictions）
create_visualizations(analysis_results, df)

### 销售热力图（按周和月）

In [68]:
print(df.columns)

Index(['销售额'], dtype='object')


In [70]:
def create_visualizations(analysis_results, df):
    # 情况 1：修正列名（根据实际列名调整）
    df = df.rename(columns={"实际日期列名": "日期"})  # 如果列名不匹配
    
    # 情况 2：生成日期列（若无日期数据）
    df["日期"] = pd.date_range(start="2023-01-01", periods=len(df), freq="D")  # 示例
    
    # 转换为日期格式并提取周/月
    df["日期"] = pd.to_datetime(df["日期"])
    df["dayofweek"] = df["日期"].dt.dayofweek  # 0=周一, 6=周日
    df["month"] = df["日期"].dt.month
    
    # 构建透视表
    pivot_table = df.pivot_table(index='dayofweek', columns='month', values='销售额', aggfunc='mean')
    
    # 绘制热力图
    plt.figure(figsize=(12, 8))
    sns.heatmap(pivot_table, annot=True, fmt='.2f', cmap='YlGnBu')
    plt.title('销售热力图（按周和月）')
    plt.xlabel('月份')
    plt.ylabel('星期')
    plt.tight_layout()
    
    # 保存时使用原始字符串避免转义错误
    plt.savefig(r"H:\期末考试\可视化\销售热力图（按周和月）.png")
    plt.close()

# 调用函数
create_visualizations(analysis_results, df)

### 油价与销售散点图

In [84]:
def create_visualizations(analysis_results, df):
    # 1. 检查正确的列名（根据数据实际列名修改）
    required_columns = ['sales']  # 假设数据中销售额列名为 'sales'
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"缺少必要的列: {missing_columns}")
        print(f"可用的列: {list(df.columns)}")
        return
    
    # 2. 绘制图表（调整列名）
    plt.figure(figsize=(10, 6))
    sample_size = min(1000, len(df))
    sample_df = df.sample(sample_size, replace=False) if len(df) > 0 else df
    
    # 若需分析油价与销售额关系，但数据中没有油价列，需补充数据或检查列名
    sns.scatterplot(x='sales', y='sales', data=sample_df)  # 示例仅用现有列
    plt.title('销售数据分布')
    plt.xlabel('销售额')
    plt.tight_layout()
    plt.savefig(r"H:\期末考试\可视化\油价与销售数据散点图.png")
    plt.close()

create_visualizations(analysis_results, df)